In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Implementation der Grafischen Benutzeroberfläche

Im folgenden Abschnitt wird eine Benutzeroberfläche für das Spiel Othello implementiert.

## Importieren der externen Abhängigkeiten

Die Grafische Benutzeroberfläche verwendet zur Darstellung des Spielzustandes, zum Anzeigen weiterer Informationen sowie für die Benutzerinteraktion die Bibliotheken `ipycanvas` und `ipywidgets`. Diese Lassen sich direkt im Jupyter Notebook verwenden.

Zusätzlich werden aus dem Paket `math` der Python Standardbibliothek die Variable `pi` sowie die Funktion `floor` benötigt.

In [ ]:
import ipycanvas
import ipywidgets
import math

## Globale Konstanten

## Canvas Initialisieren

`SHOW_FRONTIER` gibt an ob in der Visualisierung leere Felder, die an bereits gesetzte Spielsteine angrenzen hervorgehoben werden sollen.


`SHOW_POSSIBLE_MOVES` gibt ob für den aktuell ziehenden Spieler mögliche Züge visualisiert werden sollen.

In [ ]:
SHOW_FRONTIER = False
SHOW_POSSIBLE_MOVES = True

In [ ]:
CELL_SIZE = 60

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

## Widgets Initialisieren

Das `score_lbl` Widget enthält die Steinzahl beider Spieler im aktuellen Spielzustand

In [ ]:
score_lbl = ipywidgets.widgets.Label()

Das `turn_lbl` Widget nennt den Spieler, der gerade am Zug ist

In [ ]:
turn_lbl = ipywidgets.widgets.Label()

Das `output` Widget macht die Ausgabe mithilfe von `print()`, sowie die Ausgabe von Fehlermeldungen trotz der Verwendung von IPyWidgets und IPyCanvas möglich.

In [ ]:
output = ipywidgets.widgets.Output()

Die Funktion `display_board` stellt den angegebenen Spielzustand dar, indem zunächst der Canvas aktualisiert, und dann zusammen mit den Status-Widgets angezeigt wird.

In [ ]:
def display_board(state):
    update_output(state)
    display(canvas)
    display(score_lbl)
    display(turn_lbl)
    display(output)

In der Funktion `update_output` wird der Spielzustand `state` auf den Canvas gezeichnet.

In [ ]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in numpy.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5)
                               * CELL_SIZE, CELL_SIZE / 2.2, 0, 2 * math.pi)

        if SHOW_FRONTIER:
            for (x, y) in state.frontier:
                canvas[1].fill_style = 'gray'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5)
                                   * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)

        if SHOW_POSSIBLE_MOVES:
            for (x, y) in get_possible_moves(state, state.turn):
                if state.turn == BLACK:
                    canvas[1].fill_style = 'black'
                else:
                    canvas[1].fill_style = 'white'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5)
                                   * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)

    score_lbl.value = f'Black Player : {count_disks(state, BLACK)} White Player : {count_disks(state, WHITE)}'
    if state.game_over:
        turn_lbl.value = f'{get_player_string(get_winner(state))} wins'
    else:
        turn_lbl.value = f'{get_player_string(state.turn)}s Move'

Für den menschlichen Spieler ist es nötig festzustellen, ob dieser auf das Spielfeld geklickt hat, dies geschieht in der callback funktion `mouse_down` welche die x und y Koordinaten des Mausklicks relativ zum Canvas erhält. Auf Basis dieser Position wird, falls möglich, ein Zug auf das angeklickte Feld gemacht. Die Funktion wird durch den aufruf von `Canvas.on_mouse_down` bei IPyCanvas als Callback Funktion registriert.

In [ ]:
def mouse_down(x_px, y_px):
    if not state.game_over:
        with output:
            x = math.floor(x_px / CELL_SIZE)
            y = math.floor(y_px / CELL_SIZE)
            try:
                make_move(state, (x, y))
            except InvalidMoveException:
                print('Invalid Move')
            update_output(state)
            try:
                next_move(state)
            except KeyboardInterrupt:
                pass


canvas[1].on_mouse_down(mouse_down)